In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df_original = pd.read_csv('../data/_data.csv')
df = df_original.copy()

In [3]:
df.isna().sum()

Unnamed: 0                      0
ID  объявления                  0
Количество комнат            1041
Тип                             0
Метро                        1315
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    13417
Цена                            0
Телефоны                        0
Описание                        0
Ремонт                       2755
Площадь комнат, м2           8910
Балкон                       7978
Окна                         6613
Санузел                      2672
Можно с детьми/животными     6096
Дополнительно                 357
Название ЖК                 17520
Серия дома                  21205
Высота потолков, м          12162
Лифт                         5500
Мусоропровод                10522
Ссылка на объявление            0
dtype: int64

In [4]:
# 2. удаляем столбцы "Серия дома" и "Название ЖК"
df.drop(columns=['Серия дома', 'Название ЖК'], inplace=True)

In [5]:
# 4."Количество комнат" заполняем отсутствующие данные медианой по общей площади
# преобразуем столбец 'Количество комнат' в числовой тип, заменяя некорректные значения на NaN
df['Количество комнат'] = pd.to_numeric(df['Количество комнат'], errors='coerce')
# создаём два новых столбца: 'Общая площадь, м2' и 'Жилая площадь, м2'
def extract_areas(area_str):
    try:
        parts = area_str.split('/')
        total_area = float(parts[0]) if len(parts) > 0 else None
        living_area = float(parts[1]) if len(parts) > 1 else None
        return pd.Series([total_area, living_area])
    except:
        return pd.Series([None, None])
df[['Общая площадь, м2', 'Жилая площадь, м2']] = df['Площадь, м2'].apply(extract_areas)
# группируем по 'Общая площадь, м2' и вычисляем медиану для каждой группы
medians = df.groupby('Общая площадь, м2')['Количество комнат'].transform('median')
# заменяем NaN значения в 'Количество комнат' на медиану соответствующей группы
df['Количество комнат'] = df['Количество комнат'].fillna(medians)
# чтобы дробных чисел не было
df['Количество комнат'] = df['Количество комнат'].round().astype('Int64')
# Заполним NA медианой по всему столбцу (после округления)
global_med = df['Количество комнат'].median()  # это может быть float или Int64
df['Количество комнат'] = df['Количество комнат'].fillna(global_med).astype('Int64')

In [6]:
df.isna().sum()

Unnamed: 0                      0
ID  объявления                  0
Количество комнат               0
Тип                             0
Метро                        1315
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    13417
Цена                            0
Телефоны                        0
Описание                        0
Ремонт                       2755
Площадь комнат, м2           8910
Балкон                       7978
Окна                         6613
Санузел                      2672
Можно с детьми/животными     6096
Дополнительно                 357
Высота потолков, м          12162
Лифт                         5500
Мусоропровод                10522
Ссылка на объявление            0
Общая площадь, м2               0
Жилая площадь, м2            3060
dtype: int64

In [7]:
# отсутствующие значения "Метро" заменим на 'unknown'
# добавим столбец метро без времени до него
df['Станция Метро'] = df['Метро'].str.split('(').str[0].str.strip()
# заменяем  NaN на 'unknown'
df['Станция Метро'] = df['Станция Метро'].fillna('unknown')
# заменяем строки, которые пустые или содержат только пробелы на 'unknown'
df['Станция Метро'] = df['Станция Метро'].replace(r'^\s*$', 'unknown', regex=True)

In [18]:
df['Площадь, м2'].unique()

array(['200.0/20.0', '198.0/95.0/18.0', '200.0/116.0/4.0', ..., '43.1',
       '52.5/10.0', '90.0/48.2/15.5'], shape=(11255,), dtype=object)

In [19]:
missing_values_count = df['Парковка'].isna().sum()
print(missing_values_count)

13417
